In [2]:
import json
import random
import numpy as np
import pandas as pd
import os

from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh_choropleth import *

In [3]:
palette.reverse()
output_notebook()

Loading BokehJS ...

# Estimativa de tempo de espera por corrida uber, nos bairros da cidade de natal
---

In [7]:
# Time estimates

geo_json_natal = json.load(open('geojson/natal.geojson',encoding='UTF-8'))
estimativasData = pd.read_csv('data/estimates.csv', 
                              parse_dates=True, index_col=0,
                              names=['Hora_coleta', 
                                     'Bairro', 
                                    'Tipo Uber',
                                    'Tempo_espera'])

estimativasData['Tempo_espera'] = estimativasData['Tempo_espera']/60
estimativaMeanData = (estimativasData.pivot_table(index='Bairro',values='Tempo_espera')).reset_index()

In [8]:
color_mapper = LinearColorMapper(palette = palette, 
                              low=estimativaMeanData['Tempo_espera'].min(), 
                              high=estimativaMeanData['Tempo_espera'].max())

choropleth = bokeh_choropleth(data_df= estimativaMeanData,
                              geojson = geo_json_natal,
                    key_on = ['Bairro', 'name'],
                              data = 'Tempo_espera',
                              tooltips = [('Bairro', '@Bairro'), ('Tempo_espera', '@Tempo_espera')],
                              color_mapper = color_mapper)
show(choropleth)

# Estimativa de preços do uber nos bairros da cidade de natal
---

In [14]:
pricesData = pd.read_csv('data/prices.csv', names=
                         ['Tempo_coleta', 'Origem', 'Destino', 'Tipo_uber', 'Distancia', 
                          'High_price', 'Low_price', 'Duration', 'Estimate', 'Currency'])
pricesData['Mean_price'] = (pricesData['High_price'] + pricesData['Low_price'])/2.0
pricesMeanData = (pricesData.pivot_table(index=['Destino','Origem'], values='Mean_price', fill_value=0)).reset_index()

imdPricesData = pricesMeanData[pricesMeanData['Origem']=='IMD']
ectPricesData = pricesMeanData[pricesMeanData['Origem']=='EC&T']

pricesMeanData['Destino'].unique()


array(['Candelária', 'Cidade Nova', 'Cidade da Esperança', 'Igapó',
       'Nossa Senhora de Nazaré', 'Nova Descoberta', 'Planalto', 'Ribeira',
       'Santos Reis', 'Tirol'], dtype=object)

### Estimativa a partir do IMD

In [15]:
geo_10_neibhorhood = json.load(open('geojson/bairros_sorteados.geojson',encoding='UTF-8'))

color_mapper = LinearColorMapper(palette = palette, 
                              low=imdPricesData['Mean_price'].min(), 
                              high=imdPricesData['Mean_price'].max())

choropleth = bokeh_choropleth(data_df= imdPricesData,
                              geojson = geo_10_neibhorhood,
                              key_on = ['Destino', 'name'],
                              data = 'Mean_price',
                              tooltips = [('Destino', '@Destino'), ('Mean_price', '@Mean_price')],
                              color_mapper = color_mapper)

show(choropleth)

### Estimativa a partir da ECT

In [16]:
geo_10_neibhorhood = json.load(open('geojson/bairros_sorteados.geojson',encoding='UTF-8'))

color_mapper = LinearColorMapper(palette = palette, 
                              low=ectPricesData['Mean_price'].min(), 
                              high=ectPricesData['Mean_price'].max())

choropleth = bokeh_choropleth(data_df= ectPricesData,
                              geojson = geo_10_neibhorhood,
                              key_on = ['Destino', 'name'],
                              data = 'Mean_price',
                              tooltips = [('Destino', '@Destino'), ('Mean_price', '@Mean_price')],
                              color_mapper = color_mapper)

show(choropleth)